```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de datos no estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2020
Profesor: Facundo Santiago, Javier Ignacio Garcia Fronti
```


# BERT: Bidirectional Encoder Representations from Transformers

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Preparación del ambiente

Descargamos el set de datos

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv --directory-prefix ./Datasets/mascorpus/

Instalamos las librerias necesarias

In [ ]:
!pip install transformers

Cargamos el set de datos

In [3]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

## Introducción a BERT

word2vec es un modelo de tipo "context-free", lo que significa que cada palabra recibe un único vector que la representa. Esto implica que por ejemplo la palabra "banco" recibirá la misma representación en las oraciones *Los domingos no abre el banco* y *Estabamos tan cansados que nos sentamos en un banco*.

BERT, sin embargo, es un modelo contextual, lo que significa que la representación que se genera de una palabra depende del contexto en el que aparece.

### Como funciona

Si recordamos de cuando introducimos word2vec, vimos que las representaciones de las palabras se obtenian al entrenar una red neuronal en una tarea "falsa" que era predecir una palabra dado el contexto en el que aparece. Este contexto lo especificabamos como una ventana de palabras. Los modelos basados en lenguaje, llevan esta tarea un paso más adelante y tratan de predecir la siguiente palabra dada una secuencia de tokens.

En el caso de BERT, está pre-entrenado utilizando 2 tareas distintas:
 - **Masked LM:** BERT está basado en una técnica llamada Masked LM (MLM) la cual, en lugar de intentar predecir la siguiente palabra dada una secuencia de palabras, aleatoriamente enmascara palabras en la oración para luego intentar predecirlas desde el contexto. Para hacerlo utiliza el contexto completo de la oración, tanto hacia adelante como hacía atras (por esto se llama bidireccional). En practica, BERT enmascara aproximadamente el 15% de los tokens en una secuencia.
 - **NSP (Next Sentence Prediction):** Muchas tareas en NLP requieren el entendimiento de las relaciones entre varias oraciones o secuencias. BERT captura estas relaciones al estar entrenado para predecir la siguiente oración en el cuerpo. En realidad BERT utiliza 50% del tiempo efectivamente la siguiente oración para la tarea de NSP y la taguea con el token IsNext, mientras que el otro 50% utiliza una oración aleatoria del texto y la taguea con el token NoNext.

## Explorando un modelo pre-entrenado con BERT

Una de las formas más sencillas de trabajar con el modelo BERT es utilizando la libreria transformers de Hugging Face [https://huggingface.co/] la cual ofrece una forma muy conveniente de acceder a modelos de NLP en diferentes lenguajes e incluso entrenados para tareas especificas.

Podemos instalar esta libreria desde pip de la siguiente forma. Este paso ya lo realizamos en la sección de instalación de este notebook

```
pip install transformers
```

### BETO: BERT en español

Al igual que con word2vec, entrenar un modelo de lenguaje requiere de una gran cantidad de datos sumado a un poder de computo interesante (cuando BERT fué publicado en 2018, tomó 4 días entrenar el modelo usando 16 TPUs. Si se hubiera entrenado en 8 GPUs hubiera tomado entre 40–70 días).Por este motivo, utilizaremos un modelo pre-entrenado para un cuerpo de texto en español. Este modelo, BETO, fué entrenado sobre un gran corpora de texto. Pueden encontrar más información sobre el autor de este modelo en: https://github.com/dccuchile/beto 

### Tokenizers 

BERT utiliza su propio tokenizer que está basado en WordPiece. Este tokenizer tiene un vocabulario de 30.000 tokens donde cada secuencia comienza con un token especial [CLS]. Exploremos como funciona este tokenizer

In [5]:
import transformers

tokenizer = transformers.BertTokenizerFast.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', do_lower_case=True)

*Noten que el tokenizer depende del modelo que estamos utilizando*

Exploremos los tokens que genera:

In [6]:
text = tweets['TEXTO'][5]

In [8]:
tokens = tokenizer.encode(text)

¿Notan algo raro en los tokens generados?

In [ ]:
tokenizer.decode(tokens)

¿Siguen notando algo raro?

In [ ]:
[tokenizer.convert_ids_to_tokens(idx) for idx in tokens]

### Cargando nuestro modelo de BERT para español

Para cargar nuestro modelo, utilizaremos la libreria transformers nuevamente, utilizando el método from_pretrained. Este método descargará automaticamente el modelo desde el directorio de modelos de HuggingFace. Pueden ver el listado de modelos que están disponibles en este directorio en [https://huggingface.co/models]

In [17]:
model = transformers.BertForMaskedLM.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', return_dict=True)

<i>El parámetro return_dict=True hará que el modelo retorne un diccionario con los resultados en lugar de un tupla. Esto solo hará que sea más sencillo interpretar los resultados.</i>

Veamos como se comporta nuestro modelo en la tarea de predecir una palabra de un texto:

In [18]:
text = "[CLS] Cuando [MASK] contaron lo que sucedia nos quedamos helados. [SEP]"
tokens = tokenizer.tokenize(text)

Noten que el token [MASK] es la palabra que estamos intentando predecir

Necesitamos saber cual de todos los tokens que generamos es exactamente el que enmascaramos:

In [19]:
masked_indxs = [idx for idx in range(0, len(tokens)) if tokens[idx] == '[MASK]']

Corremos nuestro modelo:

In [26]:
import torch

indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
tokens_tensor = torch.tensor([indexed_tokens])

predictions = model(tokens_tensor).logits

<i>**Nota:** Ejecutar el modelo sobre nuestro texto devolverá diferente cantidad de objetos dependiendo de como se configuró la carga del modelo en el método `from_pretrained`. En nuestro caso, hemos especificado `return_dic=True` y por lo tanto la salida del médoto es un diccionario con los resultados (si no lo hubieramos especificado hubiera retornado una tupla). Dentro de lo que nos interesa a nosotros está:</i>
 - <i>**logits:** Retorna la secuencia de hidden-states en la última capa del modelo. Esto tiene tamaño (batch_size, sequence_length, hidden_size).</i>
 - <i>**hidden_states:** Retorna los hidden-states de todas las capas del modelo. 12 en el caso de BERT. Esta información solo se retorna cuando se indica `output_hidden_states=True`</i>

<i>Para más información sobre que objetos se retornan de la ejecución del modelo pueden ver la documentación del modelo BERT: https://huggingface.co/transformers/model_doc/bert.html#bertmodel</i>

Verificamos cuales son las palabras más probables:

In [27]:
for i,midx in enumerate(masked_indxs):
    idxs = torch.argsort(predictions[0,midx], descending=True)
    predicted_token = tokenizer.convert_ids_to_tokens(idxs[:5])
    print('Las 5 palabras más probables para la mascara',i,'son:',predicted_token)

Las 5 palabras más probables para la mascara 0 son: ['nos', 'me', 'les', 'le', 'supi']


### Explorando las representaciones de BERT

En esta sección exploraremos las representaciones que genera BERT. Como se mencionó, BERT generá representaciones que son dependientes del contexto, algo que lo diferencia de Word2Vec. Esto implica que por ejemplo la palabra "banco" recibirá la misma representación en las oraciones *Los domingos no abre el banco y salimos a caminar. Estabamos tan cansados que nos sentamos en un banco a ver gente pasar*. Veamos si esto es así como mencionamos explorando estas representaciones

Para hacer esto, volveremos a cargar el modelo, ahora especificando el parametro `output_hidden_states=True`. Esto hará que el modelo retorne las representaciones de todos los estados intermedios como parte de la salida:

In [47]:
model = transformers.BertForMaskedLM.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', return_dict=True, output_hidden_states=True)

In [77]:
text = "Queriamos retirar dinero del banco. Sin embargo, los domingos el banco no está abierto. Estabamos tan cansados que nos sentamos en un banco a ver gente pasar"

In [78]:
indexed_tokens = tokenizer.encode(text)

In [79]:
tokens_tensor = torch.tensor([indexed_tokens]) #Al igual que antes, siempre convertimos primero el input en un tensor

hidden_states = model(tokens_tensor).hidden_states
token_embeddings = torch.stack(hidden_states, dim=0) #hidden_states retorna una lista, transformemos esto en un tensor :)

Veamos las dimensiones de este objeto:

In [80]:
token_embeddings.shape

torch.Size([13, 1, 33, 768])

¿Que significan?
 - 13 es la cantidad de capas dentro del modelo. Si bien BERT posee 12 capas, la capa numero 0 corresponde a los inputs (entradas) del modelo y por eso vemos 13 capas finalmente
 - 1 es la cantidad de muestra en el lote (ie. batch size)
 - 33 es la cantidad de tokens que se inputaron al model
 - 768 es la cantidad de unidades de la red neuronal (units) en cada capa

Como nuestro lote/batch solo contiene una oración, podemos deshacernos de la segunda dimensión de este tensor:

In [81]:
token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings.size()

torch.Size([13, 33, 768])

Finalmente, para facilitar el entendimiento de la salida, vamos a cambiar el orden de los valores de este tensor. Recordaran que actualmente tenemos un tensor del tamaño (numero_de_capas, numero_de_tokens, features). Para el análisis que queremos realizar, sería mucho más interesante tener algo del tipo (numero_de_tokens, numero_de_capas, features). De esta forma podriamos revisar todas las representaciones de cada token más facilmente:

In [82]:
# Cambiamos las dimensiones 0 y 1
token_embeddings = token_embeddings.permute(1,0,2)
token_embeddings.size()

torch.Size([33, 13, 768])

#### ¿Entonces cuales son las representaciones?

Las representaciones son tensores del tamaño (12,768). Claramente esto es un gran progreso desde las 100 dimensiones en Word2Vec! Veamos si efectivamente las representaciones de banco son diferentes:

In [ ]:
for i, token_str in enumerate(indexed_tokens):
  print (i, tokenizer.convert_ids_to_tokens(token_str))

En este ejemplo tenemos la palabra **banco** en las posiciones 6, 14 y 27. De estas posiciones, la palabra **banco** en las posiciones 6 y 14 debería de tener un significado similar, mientra que su versión en la posición 27 debería ser distinta. Para medir esto necesitariamos comparar la similaridad de estas representaciones. Para esto podríamos utilizar una métrica como la Similaridad de coseno (cosine similarity), sin embargo necesitamos contar con un vector unidimensional. Si bien no hay una única forma de resolver este problema, una forma podría ser tomar el promedio de los valores a lo largo de las 13 capas para generar un vector unidimensional por cada token:

In [97]:
sentence_embedding = torch.mean(token_embeddings, dim=1)

In [99]:
sentence_embedding.shape

torch.Size([33, 768])

Computemos ahora las similaridad de las palabras:

In [100]:
from scipy.spatial.distance import cosine

*(...) retirar dinero del **banco**. Sin embargo, los domingos el **banco** no está abierto (...)*

In [105]:
1 - cosine(sentence_embedding[6].detach().numpy(), sentence_embedding[14].detach().numpy())

0.9034030437469482

*(...) retirar dinero del **banco** (...) nos sentamos en un **banco** a (...)*

In [96]:
1 - cosine(sentence_embedding[6].detach().numpy(), sentence_embedding[27].detach().numpy())

0.7559054493904114

Podemos ver que la similaridad entre cada una de las representaciones es distinta. Tengan en cuenta que este método no es exacto, pero de alguna forma nos da una idea y una intuición de que tan cercanas pueden ser estas representaciones. BERT es un modelo donde las representaciones dependen del contexto y por lo tanto el concepto de "similaridad" aquí es distinto e incluso podría carecer un poco de sentido.

#### De los autores de BERT

Los autores de BERT proponen una solución un poco distinta, en base a diferentes experimentos que realizaron. Una de las formas que generaron las representaciones con los mejores resultados resultó de concatenar los valores de las últimas 4 capas.

<img width="600" src='http://jalammar.github.io/images/bert-feature-extraction-contextualized-embeddings.png' />

Algo interesante que decanta de este ultimo experimento es que claramente cada uno de las diferentes capas dentro de BERT codifican diferentes aspectos de las palabras y que por lo tanto, la estrategia que mejor se adapta para generar estas representaciones más compactas depende mucho de la tarea que se esté realizando. A continuación intentaremos aplicarlo (noten que en las lineas subsiguientes habrá bastante manipulación de las formas de los vectores)

Nos quedamos con las ultimas capas de cada token:

In [145]:
slice_ = token_embeddings.narrow(1, 9, 4)

In [114]:
slice_.shape

torch.Size([33, 4, 768])

Concatenamos los valores de las últimas 4 capas:

In [151]:
concatenated_tensor = slice_.reshape(33, 4*768)

In [152]:
concatenated_tensor.shape

torch.Size([33, 3072])

Calculamos la similaridad:

In [153]:
1 - cosine(concatenated_tensor[6].detach().numpy(), concatenated_tensor[14].detach().numpy())

0.848562479019165

In [154]:
1 - cosine(concatenated_tensor[6].detach().numpy(), concatenated_tensor[27].detach().numpy())

0.6665388941764832